In [2]:
import sys
from pathlib import Path

root_dir = Path().absolute().parent.parent
sys.path.append(str(root_dir))

In [3]:
# Warning control
import os
import warnings

import controlflow as cf
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

load_dotenv()

True

In [4]:
import requests
from bs4 import BeautifulSoup
from controlflow import tool
from pydantic import BaseModel, Field


class Event(BaseModel):
    title: str = Field(..., description="The title of the event")
    summary: str = Field(..., description="A short summary of the event")
    detail_url: str = Field(..., description="The URL of the event detail page")

scrape_events = cf.Task(
    objective="Scrape events from the Siegessäule website",
    instructions="Identify the events on the page of the provided URL and return the \
                  title, summary, and detail URL for each event",
    context={"url": "https://www.siegessaeule.de/en/events/?date=2025-01-21"},
    result_type=Event,
)

@tool
def scrape_main_content(url: str) -> str:
    """
    Scrape the main content from the provided URL.
    Returns the HTML content as a string.
    """
    # Send GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    
    # Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the main content area (you may need to adjust these selectors based on the actual website structure)
    main_content = soup.find('main') or soup.find('div', class_='content') or soup.find('div', id='content')
    
    # If no main content found, use the entire body
    if not main_content:
        main_content = soup.body
    
    # Return the HTML content as a string
    return str(main_content)
    

@cf.flow
def research_proposal_flow():

    # Task 1: Get the research topic from the user
    user_input = cf.Task(
        "Work with the user to choose a research topic",
        interactive=True,
    )
    
    # Task 2: Generate a structured research proposal
    proposal = cf.run(
        "Generate a structured research proposal",
        result_type=ResearchProposal,
        depends_on=[user_input]
    )
    
    return proposal

def main():
    
    result = research_proposal_flow()

    print(result.model_dump_json(indent=2))

main()

🤖 Agent: Hello! I'm here to help you choose a research topic. To get started, could you let me know the general 
field or area you're interested in researching? For example, are you interested in technology, health, social 
sciences, or something else? Feel free to share any specific ideas or themes you have in mind as well!
Type your response:

🤖 Agent: No worries! Let's try to narrow it down together. Here are a few broad areas you might consider:

1. **Technology**: Topics related to artificial intelligence, cybersecurity, renewable energy, etc.
2. **Health**: Research on mental health, nutrition, public health policies, etc.
3. **Social Sciences**: Topics in sociology, psychology, education, etc.
4. **Environment**: Climate change, conservation, sustainable practices, etc.
5. **Economics**: Global markets, economic policies, financial technologies, etc.

Do any of these areas sound interesting to you, or is there another field you're curious about?
Type your response:

{
  "title": "The Influence of Prime Numbers on the Performance of Artificial Intelligence",
  "abstract": "This research proposal aims to explore the impact of prime numbers on the performance of artificial intelligence (AI) systems. While prime numbers have been extensively studied in mathematics and cryptography, their influence on AI algorithms and performance metrics remains relatively unexplored. This study will investigate the potential applications of prime numbers in optimizing AI processes, enhancing algorithm efficiency, and improving computational outcomes. Through theoretical analysis, simulations, and practical experiments, this research seeks to uncover new insights that could lead to innovative approaches in AI development and deployment.",
  "key_points": [
    "Explore the mathematical properties of prime numbers relevant to AI.",
    "Examine the role of prime numbers in algorithm optimization.",
    "Analyze the impact of prime numbers on AI performance metrics.",
 